In [ ]:
!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install datasets -q

In [ ]:
import pandas as pd
import torch
import json

from huggingface_hub import hf_hub_download

from math import ceil

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Ładowanie Bielika

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda"
model_name = 'speakleash/Bielik-7B-v0.1'


temperature = 1.0
max_tokens = 500
top_k = 200
top_p = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
      load_in_8bit=True,
      # bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            torch_dtype=torch.bfloat16,
                                            quantization_config=quantization_config
                                            )

model.generation_config.pad_token_id = tokenizer.pad_token_id


`low_cpu_mem_usage` was None, now set to True since model is quantized.


model-00002-of-00003.safetensors:   8%|8         | 409M/5.00G [00:00<?, ?B/s]

In [ ]:
# TODO To Rewrite into create_input(), and generate()

def generate(prompt, system = None):

  messages = []

  if system:
    messages.append( {"role": "system", "content": system })

  messages.append( {"role": "user", "content": prompt })


  tokenizer_output = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True)

  if torch.cuda.is_available():
    model_input_ids = tokenizer_output.input_ids.to(device)
    model_attention_mask = tokenizer_output.attention_mask.to(device)

  else:
    model_input_ids = tokenizer_output.input_ids
    model_attention_mask = tokenizer_output.attention_mask


  # generowanie odpowiedzi
  outputs = model.generate(model_input_ids,
                           attention_mask=model_attention_mask,
                           streamer = streamer,
                           max_new_tokens=max_tokens,
                           do_sample=True if temperature else False,
                           temperature = temperature,
                           top_k = top_k,
                           top_p = top_p)

  # # zapis do pliku
  # filename = f"{str(uuid.uuid4())[:8]}.json"

  # with open(filename, "w") as file:
  #     content = {
  #         "prompt": messages,
  #          "output": tokenizer.batch_decode(outputs, skip_special_tokens=False)
  #     }
  #     json.dump(content, file, ensure_ascii=False, indent=4)




In [ ]:
temperature = 0
max_tokens = 1000
top_k = 200
top_p = 1


query = "Z Ilu kości składa się czaszka?"
prompt = f"Odpowiedz zwięźle na pytanie: '{query}'"
generate(prompt)

## Ładowanie Danych Q&A (Poquad)

In [ ]:
hf_hub_download(repo_id="clarin-pl/poquad", filename="poquad-train.json", repo_type="dataset", local_dir="./data")

In [ ]:
def read_poquad_data(filepath):
        whole_data = []
        with open(filepath, encoding="utf-8") as f:
            squad = json.load(f)
            id_ = 0
            for example in squad["data"]:
                title = example.get("title", "")
                for paragraph in example["paragraphs"]:
                    context = paragraph["context"]
                    for qa in paragraph["qas"]:
                        question = qa["question"]

                        answer_starts = [answer["answer_start"] for answer in qa["answers"]] if "answers" in qa else []

                        answers = [answer["text"] for answer in qa["answers"]] if "answers" in qa else []
                        is_impossible = qa["is_impossible"]

                        id_ += 1
                        whole_data.append({
                            "id": id_,
                            "title": title,
                            "context": context,
                            "question": question,
                            "is_impossible" : is_impossible,
                            "answers": {
                                "answer_start": answer_starts,
                                "text": answers,
                            },
                        })

        return pd.DataFrame(whole_data).set_index("id")

In [ ]:
df = read_poquad_data("./data/poquad-train.json")

df["target"] = df["answers"].apply(lambda x: "Odpowiedź: " + x["text"][0] if x["text"] else "Odpowiedź niemożliwa!")

In [ ]:
def print_using_shorter_lines(words_list, word_limit):
  n_fragments = ceil(len(words_list)/word_limit)

  for i in range(n_fragments):
    print(" ".join(words_list[12*i:12*(i+1)]))


def print_poquad_row(df, i, n_words_in_line=12):
  print("Kontekst:")

  context = df.loc[i, "context"].split(" ")
  print_using_shorter_lines(context, n_words_in_line)

  print("\nPytanie:")
  question = df.loc[i, "question"].split(" ")
  print_using_shorter_lines(question, n_words_in_line)

  print("\nOczekiwana odpowiedź:")
  answer = df.loc[i, "target"].split(" ")
  print_using_shorter_lines(answer, n_words_in_line)

def create_prompt_from_row(df, i):
  return "Kontekst: " + df.loc[i, ["context", "question"]].str.cat(sep="\nPytanie: ")




---

## Przykładowe generowanie odpowiedzi

In [ ]:
print_poquad_row(df, 1)

In [ ]:
prompt_possible = create_prompt_from_row(df, 1)

In [ ]:
system_prompt = """Jesteś chatbotem odpowiadającym zwięźle na pytania na podstawie podanego kontekstu. Niektóre pytania mogą być niemożliwe, w takim wypadku musisz wstrzymać się od odpowiedzi. Twoje odpowiedzi mają przyjąć formę:
"Odpowiedź: 'treść odpowiedzi'"
jeśli treść odpowiedź istnieje w tekście
lub
"Brak Odpowiedzi!"
jeśli nie da się odpowiedzieć na pytanie na podstawie kontekstu."""

generate(prompt=prompt_possible, system=system_prompt)

Model Generuje przesadnie rozległy tekst.

---

In [ ]:
print_poquad_row(df, 8)

In [ ]:
prompt_impossible = create_prompt_from_row(df, 8)

In [ ]:
generate(prompt=prompt_impossible, system=system_prompt)

Model Halucynuje.

---

## Generowanie przesłanek


In [ ]:
print_poquad_row(df, 26070)

In [ ]:
system_prompt = f"""Jesteś chatbotem przeprowadzającym wnioskowanie mające na celu znalezienie fragmentu kontekstu odpowiadającego na pytanie. Niektóre pytania mogą być niemożliwe, w takim wypadku musisz wykazać, że odpowiedź jest niemożliwa i napisać 'Odpowiedź niemożliwa!'. Przykładowa forma odpowiedzi:
"Wnioskowanie: 'treść przesłanki;
Odpowiedź: 'treść odpowiedzi - dlaczego odpowiedź jest możliwa'"
jeśli treść odpowiedź istnieje w tekście
lub
"Wnioskowanie: 'treść przesłanki - dlaczego odpowiedź jest niemożliwa'
Odpowiedź niemożliwa!"
jeśli nie da się odpowiedzieć na pytanie na podstawie kontekstu.

Przykład 1:
Kontekst: {df.loc[26070, "context"]}
Pytanie: {df.loc[26070, "question"]}

Wnioskowanie: Pytano o powód zasiedlenia Lubina. Wyrażenie 'związane jest' wskazuje na wpływ przecięcia się szlaków kupiekcich z powstaniem Lubina.
{df.loc[26070, "target"]}

Przykład 2:
Kontekst: {df.loc[25251, "context"]}
Pytanie: {df.loc[25251, "question"]}

Wnioskowanie: W tekście liczba głosów dotyczy wyborów parlamentarnych. Pytano o wybory na prezydenta stolicy Polski, a więc brak informacji w tekście.
{df.loc[25251, "target"]}

Przykład 3:
Kontekst: {df.loc[7900, "context"]}
Pytanie: {df.loc[7900, "question"]}

Wnioskowanie: W tekście informacja o liczbie posłów pojawia się w zdaniu: 'Do Sejmu Ustawodawczego weszło razem 36 posłów PPSD i PPS'. Pytano o liczbę posłów PPSD. W tekście podano liczbę posłów PPSD i PPS razem wziętych. Nie da się podać tylko liczby posłów PPSD.
{df.loc[7900, "target"]}
"""

generate(prompt=prompt_possible, system=system_prompt)

In [ ]:
generate(prompt=create_prompt_from_row(df, 56607), system=system_prompt)

In [ ]:
df[df["target"] == "Odpowiedź niemożliwa!"]